"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.
!pip install wget
!pip install git+https://github.com/NVIDIA/apex.git
!pip install nemo-toolkit
!pip install nemo-asr
!pip install unidecode

!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/master/examples/asr/configs/quartznet_speech_commands_3x1_v1.yaml
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/master/examples/asr/configs/quartznet_speech_commands_3x1_v2.yaml

In [1]:
# Import some necessary libraries
import os
import argparse

import copy
import math
import os
import glob
from functools import partial
from datetime import datetime
from ruamel.yaml import YAML

# Introduction

This Speech Command recognition tutorial is based on the QuartzNet model from the paper "[QuartzNet: Deep Automatic Speech Recognition with 1D Time-Channel Separable Convolutions](https://arxiv.org/pdf/1910.10261.pdf)" with a modified decoder head to suit classification tasks.

The notebook will follow the steps below:

 - Dataset preparation: Preparing Google Speech Commands dataset

 - Audio preprocessing (feature extraction): signal normalization, windowing, (log) spectrogram (or mel scale spectrogram, or MFCC)

 - Data augmentation using SpecAugment "[SpecAugment: A Simple Data Augmentation Method for Automatic Speech Recognition](https://arxiv.org/abs/1904.08779)" to increase number of data samples.
 
 - Develop a small Neural classification model which can be trained efficiently.
 
 - Model training on the Google Speech Commands dataset in NeMo.
 
 - Evaluation of error cases of the model by audibly hearing the samples

# This is where the Google Speech Commands directory will be placed.
# Change this if you don't want the data to be extracted in the current directory.
# Select the version of the dataset required as well (can be 1 or 2)
DATASET_VER = 2
data_dir = './google_dataset_v{0}/'.format(DATASET_VER)


In [2]:
data_dir = '/home/fjia/data/freesound_resampled'

# Data Preparation

We will be using the open source Google Speech Commands Dataset (we will use V1 of the dataset for the tutorial, but require very minor changes to support V2 dataset). These scripts below will download the dataset and convert it to a format suitable for use with nemo_asr

## Download the dataset

The dataset must be prepared using the scripts provided under the `{NeMo root directory}/scripts` sub-directory. 

Run the following command below to download the training script and execute it.

**NOTE**: You should have at least 4GB of disk space available if you’ve used --data_version=1; and at least 6GB if you used --data_version=2. Also, it will take some time to download and process, so go grab a coffee.

**NOTE**: You may additionally pass a `--rebalance` flag at the end of the `process_speech_commands_data.py` script to rebalance the class samples in the manifest.

!mkdir {data_dir}
!python process_speech_commands_data.py --data_root={data_dir} --data_version={DATASET_VER}
print("Dataset ready !")

## Prepare the path to manifest files

dtaset_path = 'google_speech_recognition_v{0}'.format(DATASET_VER)
dataset_basedir = os.path.join(data_dir, dtaset_path)

train_dataset = os.path.join(dataset_basedir, 'train_manifest.json')
val_dataset = os.path.join(dataset_basedir, 'validation_manifest.json')
test_dataset = os.path.join(dataset_basedir, 'validation_manifest.json')

# COMBO 2 
## Background + Speech Command 57k, 7k, 7k

In [3]:

dataset_basedir = data_dir

train_dataset = './manifest/background_training_manifest.json,./manifest/2balanced_sc_train_manifest.json'
# test_dataset  = './manifest/background_testing_manifest.json,./manifest/2balanced_sc_test_manifest.json'
test_dataset  = './manifest/all_test.json'

## Read a few rows of the manifest file 

Manifest files are the data structure used by NeMo to declare a few important details about the data :

1) `audio_filepath`: Refers to the path to the raw audio file <br>
2) `command`: The class label (or speech command) of this sample <br>
3) `duration`: The length of the audio file, in seconds.

In [4]:
!tail -n 10 {test_dataset}

{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/dfdabe19_nohash_0.wav", "duration": 0.810625, "label": "commands"}
{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/37fc5d97_nohash_0.wav", "duration": 1.0, "label": "commands"}
{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/964c7c9e_nohash_0.wav", "duration": 0.97525, "label": "commands"}
{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/cd85758f_nohash_0.wav", "duration": 1.0, "label": "commands"}
{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/8494fba8_nohash_2.wav", "duration": 1.0, "label": "commands"}
{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/8494fba8_nohash_0.wav", "duration": 1.0, "label": "commands"}
{"audio_filepath": "./google_dataset_v2/google_speech_recognition_v2/happy/1f3bece8_nohash_1.wav", "duration": 1.0, "label": "commands"}
{"audio_filepath": "./google_dat

# Training - Preparation

We will be training a QuartzNet model from the paper "[QuartzNet: Deep Automatic Speech Recognition with 1D Time-Channel Separable Convolutions](https://arxiv.org/pdf/1910.10261.pdf)". The benefit of QuartzNet over JASPER models is that they use Separable Convolutions, which greatly reduce the number of parameters required to get good model accuracy.

QuartzNet models generally follow the model definition pattern QuartzNet-[BxR], where B is the number of blocks and R is the number of convolutional sub-blocks. Each sub-block contains a 1-D masked convolution, batch normalization, ReLU, and dropout:


In [5]:
DATASET_VER = 1
COMBO_VERSION = 'combo_balanced_sc_bg_combine_test'

In [6]:
# Lets load the config file for the QuartzNet 3x1 model
# Here we will be using separable convolutions
# with 3 blocks (k=3 repeated once r=1 from the picture above)
yaml = YAML(typ="safe")
with open("configs/quartznet_vad_3x1_v{0}.yaml".format(DATASET_VER)) as f:
    jasper_params = yaml.load(f)

# Pre-define a set of labels that this model must learn to predict
labels = jasper_params['labels']

# Get the sampling rate of the data
sample_rate = jasper_params['sample_rate']

In [7]:
# Import NeMo core functionality
# NeMo's "core" package
import nemo
# NeMo's ASR collection
import nemo.collections.asr as nemo_asr
# NeMo's learning rate policy
from nemo.utils.lr_policies import CosineAnnealing
from nemo.collections.asr.helpers import (
    monitor_classification_training_progress,
    process_classification_evaluation_batch,
    process_classification_evaluation_epoch,
)
from nemo.collections.asr.metrics import classification_accuracy

logging = nemo.logging

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

/home/fjia/anaconda3/envs/vad/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/fjia/anaconda3/envs/vad/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' re

## Define some model hyper parameters

In [8]:
# Lets define some hyper parameters
lr = 0.05
num_epochs = 5 #5
batch_size = 128
weight_decay = 0.001

## Define the NeMo components

In [9]:
# Create a Neural Factory
# It creates log files and tensorboard writers for us among other functions
neural_factory = nemo.core.NeuralModuleFactory(
    log_dir='./{0}/quartznet-3x1-v{1}'.format(dataset_basedir, COMBO_VERSION),
    create_tb_writer=True)
tb_writer = neural_factory.tb_writer

In [10]:
# Check if data augmentation such as white noise and time shift augmentation should be used
audio_augmentor = jasper_params.get('AudioAugmentor', None)

# Build the input data layer and the preprocessing layers for the train set
train_data_layer = nemo_asr.AudioToSpeechLabelDataLayer(
    manifest_filepath=train_dataset,
    labels=labels,
    sample_rate=sample_rate,
    batch_size=batch_size,
    num_workers=os.cpu_count(),
    augmentor=audio_augmentor,
    shuffle=True
)

 # Build the input data layer and the preprocessing layers for the test set
eval_data_layer = nemo_asr.AudioToSpeechLabelDataLayer(
    manifest_filepath=test_dataset,
    sample_rate=sample_rate,
    labels=labels,
    batch_size=batch_size,
    num_workers=os.cpu_count(),
    shuffle=False,
)

# We will convert the raw audio data into MelSpectrogram Features to feed as input to our model
data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor(
    sample_rate=sample_rate, **jasper_params["AudioToMelSpectrogramPreprocessor"],
)

# Compute the total number of samples and the number of training steps per epoch
N = len(train_data_layer)
steps_per_epoch = math.ceil(N / float(batch_size) + 1)

logging.info("Steps per epoch : {0}".format(steps_per_epoch))
logging.info('Have {0} examples to train on.'.format(N))

# Here we begin defining all of the augmentations we want
# We will pad the preprocessed spectrogram image to have a certain number of timesteps
# This centers the generated spectrogram and adds black boundaries to either side
# of the padded image.
crop_pad_augmentation = nemo_asr.CropOrPadSpectrogramAugmentation(audio_length=128)

# We also optionally add `SpecAugment` augmentations based on the config file
# SpecAugment has various possible augmentations to the generated spectrogram
# 1) Frequency band masking
# 2) Time band masking
# 3) Rectangular cutout
spectr_augment_config = jasper_params.get('SpectrogramAugmentation', None)
if spectr_augment_config:
    data_spectr_augmentation = nemo_asr.SpectrogramAugmentation(**spectr_augment_config)

# Build the QuartzNet Encoder model
# The config defines the layers as a list of dictionaries
# The first and last two blocks are not considered when we say QuartzNet-[BxR]
# B is counted as the number of blocks after the first layer and before the penultimate layer.
# R is defined as the number of repetitions of each block in B.
# Note: We can scale the convolution kernels size by the float parameter `kernel_size_factor`
jasper_encoder = nemo_asr.JasperEncoder(**jasper_params["JasperEncoder"])

# We then define the QuartzNet decoder.
# This decoder head is specialized for the task for classification, such that it
# accepts a set of `N-feat` per timestep of the model, and averages these features
# over all the timesteps, before passing a Linear classification layer on those features.
jasper_decoder = nemo_asr.JasperDecoderForClassification(
    feat_in=jasper_params["JasperEncoder"]["jasper"][-1]["filters"],
    num_classes=len(labels),
    **jasper_params['JasperDecoderForClassification'],
)

# We can easily apply cross entropy loss to train this model
ce_loss = nemo_asr.CrossEntropyLossNM()

[NeMo I 2020-04-29 10:23:50 collections:222] Filtered duration for loading collection is 7.351812.
[NeMo I 2020-04-29 10:23:51 collections:222] Filtered duration for loading collection is 1.173313.
[NeMo I 2020-04-29 10:23:51 features:144] PADDING: 16
[NeMo I 2020-04-29 10:23:51 features:152] STFT using conv
[NeMo I 2020-04-29 10:23:53 <ipython-input-10-242cb97ccf7d>:34] Steps per epoch : 894
[NeMo I 2020-04-29 10:23:53 <ipython-input-10-242cb97ccf7d>:35] Have 114200 examples to train on.


In [11]:
# Lets print out the number of parameters of this model
logging.info('================================')
logging.info(f"Number of parameters in encoder: {jasper_encoder.num_weights}")
logging.info(f"Number of parameters in decoder: {jasper_decoder.num_weights}")
logging.info(
    f"Total number of parameters in model: " f"{jasper_decoder.num_weights + jasper_encoder.num_weights}"
)
logging.info('================================')

[NeMo I 2020-04-29 10:23:53 <ipython-input-11-6805b5462cf6>:2] ================================
[NeMo I 2020-04-29 10:23:53 <ipython-input-11-6805b5462cf6>:3] Number of parameters in encoder: 73344
[NeMo I 2020-04-29 10:23:53 <ipython-input-11-6805b5462cf6>:4] Number of parameters in decoder: 258
[NeMo I 2020-04-29 10:23:53 <ipython-input-11-6805b5462cf6>:6] Total number of parameters in model: 73602
[NeMo I 2020-04-29 10:23:53 <ipython-input-11-6805b5462cf6>:8] ================================


## Compile the Training Graph for NeMo

In [12]:
# Now we have all of the components that are required to build the NeMo execution graph!
## Build the training data loaders and preprocessors first
audio_signal, audio_signal_len, labels, label_len = train_data_layer()
processed_signal, processed_signal_len = data_preprocessor(input_signal=audio_signal, length=audio_signal_len)
processed_signal, processed_signal_len = crop_pad_augmentation(
    input_signal=processed_signal,
    length=audio_signal_len
)

## Augment the dataset for training
if spectr_augment_config:
    processed_signal = data_spectr_augmentation(input_spec=processed_signal)

## Define the model
encoded, encoded_len = jasper_encoder(audio_signal=processed_signal, length=processed_signal_len)
decoded = jasper_decoder(encoder_output=encoded)

## Obtain the train loss
train_loss = ce_loss(logits=decoded, labels=labels)


## Compile the Test Graph for NeMo

In [13]:
# Now we build the test graph in a similar way, reusing the above components
## Build the test data loader and preprocess same way as train graph
## But note, we do not add the spectrogram augmentation to the test graph !
test_audio_signal, test_audio_signal_len, test_labels, test_label_len = eval_data_layer()
test_processed_signal, test_processed_signal_len = data_preprocessor(
    input_signal=test_audio_signal, length=test_audio_signal_len
)
test_processed_signal, test_processed_signal_len = crop_pad_augmentation(
    input_signal=test_processed_signal, length=test_processed_signal_len
)

# Pass the test data through the model encoder and decoder
test_encoded, test_encoded_len = jasper_encoder(
    audio_signal=test_processed_signal, length=test_processed_signal_len
)
test_decoded = jasper_decoder(encoder_output=test_encoded)

# Compute test loss for visualization
test_loss = ce_loss(logits=test_decoded, labels=test_labels)

## Setting up callbacks for training and test set evaluation, and checkpoint saving

In [14]:
# Now that we have our training and evaluation graphs built,
# we can focus on a few callbacks to help us save the model checkpoints
# during training, as well as display train and test metrics

# Callbacks needed to print train info to console and Tensorboard
train_callback = nemo.core.SimpleLossLoggerCallback(
    # Notice that we pass in loss, predictions, and the labels.
    # Of course we would like to see our training loss, but we need the
    # other arguments to calculate the accuracy.
    tensors=[train_loss, decoded, labels],
    # The print_func defines what gets printed.
    print_func=partial(monitor_classification_training_progress, eval_metric=None),
    get_tb_values=lambda x: [("loss", x[0])],
    tb_writer=neural_factory.tb_writer,
)

# Callbacks needed to print test info to console and Tensorboard
tagname = 'TestSet'
eval_callback = nemo.core.EvaluatorCallback(
    eval_tensors=[test_loss, test_decoded, test_labels],
    user_iter_callback=partial(process_classification_evaluation_batch, top_k=1),
    user_epochs_done_callback=partial(process_classification_evaluation_epoch, eval_metric=1, tag=tagname),
    eval_step=200,  # How often we evaluate the model on the test set
    tb_writer=neural_factory.tb_writer,
)

# Callback to save model checkpoints
chpt_callback = nemo.core.CheckpointCallback(
    folder=neural_factory.checkpoint_dir,
    step_freq=1000,
)

# Prepare a list of checkpoints to pass to the engine
callbacks = [train_callback, eval_callback, chpt_callback]

# Training the model

Even with such a small model (77k parameters), and just 5 epochs (should take just a few minutes to train), you should be able to get a test set accuracy score in the range 85 - 90%. Not bad for a 30 (v1) or 35 (v2) way classification problem !

Experiment with increasing the number of epochs or with batch size to see how much you can improve the score!

In [15]:
import time

In [16]:
start = time.time()

# Now we have all the components required to train the model
# Lets define a learning rate schedule

# Define a learning rate schedule
lr_policy = CosineAnnealing(
    total_steps=num_epochs * steps_per_epoch,
    warmup_ratio=0.05,
    min_lr=0.001,
)

logging.info(f"Using `{lr_policy}` Learning Rate Scheduler")

# Finally, lets train this model !
neural_factory.train(
    tensors_to_optimize=[train_loss],
    callbacks=callbacks,
    lr_policy=lr_policy,
    optimizer="novograd",
    optimization_params={
        "num_epochs": num_epochs,
        "max_steps": None,
        "lr": lr,
        "momentum": 0.95,
        "betas": (0.98, 0.5),
        "weight_decay": weight_decay,
        "grad_norm_clip": None,
    },
    batches_per_step=1,
)

end = time.time()

[NeMo I 2020-04-29 10:23:53 <ipython-input-16-689768946717>:13] Using `<nemo.utils.lr_policies.CosineAnnealing object at 0x7fbbd0259050>` Learning Rate Scheduler
[NeMo I 2020-04-29 10:23:53 callbacks:187] Starting .....
[NeMo I 2020-04-29 10:23:53 callbacks:359] Found 2 modules with weights:
[NeMo I 2020-04-29 10:23:53 callbacks:361] JasperDecoderForClassification
[NeMo I 2020-04-29 10:23:53 callbacks:361] JasperEncoder
[NeMo I 2020-04-29 10:23:53 callbacks:362] Total model parameters: 73602
[NeMo I 2020-04-29 10:23:53 callbacks:311] Found checkpoint folder .//home/fjia/data/freesound_resampled/quartznet-3x1-vcombo_balanced_sc_bg_combine_test/checkpoints. Will attempt to restore checkpoints from it.


[NeMo W 2020-04-29 10:23:53 callbacks:328] For module JasperDecoderForClassification, no file matches  in .//home/fjia/data/freesound_resampled/quartznet-3x1-vcombo_balanced_sc_bg_combine_test/checkpoints
[NeMo W 2020-04-29 10:23:53 callbacks:330] Checkpoint folder .//home/fjia/data/freesound_resampled/quartznet-3x1-vcombo_balanced_sc_bg_combine_test/checkpoints was present but nothing was restored. Continuing training from random initialization.


[NeMo I 2020-04-29 10:23:53 callbacks:199] Starting epoch 0
[NeMo I 2020-04-29 10:23:54 callbacks:224] Step: 0
[NeMo I 2020-04-29 10:23:54 helpers:104] Loss: 0.7197867035865784
[NeMo I 2020-04-29 10:23:54 helpers:110] training_batch_top@1:  47.6562
[NeMo I 2020-04-29 10:23:54 callbacks:239] Step time: 0.20113658905029297 seconds
[NeMo I 2020-04-29 10:23:54 callbacks:445] Doing Evaluation ..............................


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


[NeMo I 2020-04-29 10:23:56 helpers:273] ==========>>>>>>Evaluation Loss TestSet: 0.6934638023376465
[NeMo I 2020-04-29 10:23:56 helpers:275] ==========>>>>>>Evaluation Accuracy Top@1 TestSet: 49.4914
[NeMo I 2020-04-29 10:23:56 callbacks:450] Evaluation time: 2.7741281986236572 seconds
[NeMo I 2020-04-29 10:23:58 callbacks:224] Step: 25
[NeMo I 2020-04-29 10:23:58 helpers:104] Loss: 0.14671579003334045
[NeMo I 2020-04-29 10:23:58 helpers:110] training_batch_top@1:  96.0938
[NeMo I 2020-04-29 10:23:58 callbacks:239] Step time: 0.0547490119934082 seconds
[NeMo I 2020-04-29 10:23:59 callbacks:224] Step: 50
[NeMo I 2020-04-29 10:23:59 helpers:104] Loss: 0.13893286883831024
[NeMo I 2020-04-29 10:23:59 helpers:110] training_batch_top@1:  94.5312
[NeMo I 2020-04-29 10:23:59 callbacks:239] Step time: 0.054253339767456055 seconds
[NeMo I 2020-04-29 10:24:00 callbacks:224] Step: 75
[NeMo I 2020-04-29 10:24:00 helpers:104] Loss: 0.2059304118156433
[NeMo I 2020-04-29 10:24:00 helpers:110] trainin

...

[NeMo I 2020-04-29 10:26:04 callbacks:224] Step: 1900
[NeMo I 2020-04-29 10:26:04 helpers:104] Loss: 0.11703500896692276
[NeMo I 2020-04-29 10:26:04 helpers:110] training_batch_top@1:  95.3125
[NeMo I 2020-04-29 10:26:04 callbacks:239] Step time: 0.054168701171875 seconds
[NeMo I 2020-04-29 10:26:06 callbacks:224] Step: 1925
[NeMo I 2020-04-29 10:26:06 helpers:104] Loss: 0.06961838901042938
[NeMo I 2020-04-29 10:26:06 helpers:110] training_batch_top@1:  98.4375
[NeMo I 2020-04-29 10:26:06 callbacks:239] Step time: 0.05392885208129883 seconds
[NeMo I 2020-04-29 10:26:07 callbacks:224] Step: 1950
[NeMo I 2020-04-29 10:26:07 helpers:104] Loss: 0.06956960260868073
[NeMo I 2020-04-29 10:26:07 helpers:110] training_batch_top@1:  98.4375
[NeMo I 2020-04-29 10:26:07 callbacks:239] Step time: 0.05443620681762695 seconds
[NeMo I 2020-04-29 10:26:08 callbacks:224] Step: 1975
[NeMo I 2020-04-29 10:26:08 helpers:104] Loss: 0.12177383899688721
[NeMo I 2020-04-29 10:26:08 helpers:110] training_batch_

[NeMo I 2020-04-29 10:26:46 helpers:104] Loss: 0.21195298433303833
[NeMo I 2020-04-29 10:26:46 helpers:110] training_batch_top@1:  90.6250
[NeMo I 2020-04-29 10:26:46 callbacks:239] Step time: 0.0598597526550293 seconds
[NeMo I 2020-04-29 10:26:47 callbacks:224] Step: 2550
[NeMo I 2020-04-29 10:26:47 helpers:104] Loss: 0.08120717853307724
[NeMo I 2020-04-29 10:26:47 helpers:110] training_batch_top@1:  98.4375
[NeMo I 2020-04-29 10:26:47 callbacks:239] Step time: 0.05080556869506836 seconds
[NeMo I 2020-04-29 10:26:48 callbacks:224] Step: 2575
[NeMo I 2020-04-29 10:26:48 helpers:104] Loss: 0.15907777845859528
[NeMo I 2020-04-29 10:26:48 helpers:110] training_batch_top@1:  94.5312
[NeMo I 2020-04-29 10:26:48 callbacks:239] Step time: 0.05122995376586914 seconds
[NeMo I 2020-04-29 10:26:50 callbacks:224] Step: 2600
[NeMo I 2020-04-29 10:26:50 helpers:104] Loss: 0.0868486613035202
[NeMo I 2020-04-29 10:26:50 helpers:110] training_batch_top@1:  96.8750
[NeMo I 2020-04-29 10:26:50 callbacks:

[NeMo I 2020-04-29 10:27:27 callbacks:239] Step time: 0.05430293083190918 seconds
[NeMo I 2020-04-29 10:27:28 callbacks:224] Step: 3150
[NeMo I 2020-04-29 10:27:28 helpers:104] Loss: 0.1677408665418625
[NeMo I 2020-04-29 10:27:28 helpers:110] training_batch_top@1:  94.5312
[NeMo I 2020-04-29 10:27:28 callbacks:239] Step time: 0.05421328544616699 seconds
[NeMo I 2020-04-29 10:27:30 callbacks:224] Step: 3175
[NeMo I 2020-04-29 10:27:30 helpers:104] Loss: 0.1150323674082756
[NeMo I 2020-04-29 10:27:30 helpers:110] training_batch_top@1:  96.0938
[NeMo I 2020-04-29 10:27:30 callbacks:239] Step time: 0.050679922103881836 seconds
[NeMo I 2020-04-29 10:27:31 callbacks:224] Step: 3200
[NeMo I 2020-04-29 10:27:31 helpers:104] Loss: 0.09516698122024536
[NeMo I 2020-04-29 10:27:31 helpers:110] training_batch_top@1:  98.4375
[NeMo I 2020-04-29 10:27:31 callbacks:239] Step time: 0.054258108139038086 seconds
[NeMo I 2020-04-29 10:27:31 callbacks:445] Doing Evaluation ..............................
[N

[NeMo I 2020-04-29 10:28:11 callbacks:224] Step: 3775
[NeMo I 2020-04-29 10:28:11 helpers:104] Loss: 0.09238077700138092
[NeMo I 2020-04-29 10:28:11 helpers:110] training_batch_top@1:  95.3125
[NeMo I 2020-04-29 10:28:11 callbacks:239] Step time: 0.05410361289978027 seconds
[NeMo I 2020-04-29 10:28:12 callbacks:224] Step: 3800
[NeMo I 2020-04-29 10:28:12 helpers:104] Loss: 0.07627440989017487
[NeMo I 2020-04-29 10:28:12 helpers:110] training_batch_top@1:  97.6562
[NeMo I 2020-04-29 10:28:12 callbacks:239] Step time: 0.052138328552246094 seconds
[NeMo I 2020-04-29 10:28:12 callbacks:445] Doing Evaluation ..............................
[NeMo I 2020-04-29 10:28:15 helpers:273] ==========>>>>>>Evaluation Loss TestSet: 0.049037039279937744
[NeMo I 2020-04-29 10:28:15 helpers:275] ==========>>>>>>Evaluation Accuracy Top@1 TestSet: 98.6437
[NeMo I 2020-04-29 10:28:15 callbacks:450] Evaluation time: 2.6959190368652344 seconds
[NeMo I 2020-04-29 10:28:16 callbacks:224] Step: 3825
[NeMo I 2020-0

[NeMo I 2020-04-29 10:28:52 callbacks:224] Step: 4400
[NeMo I 2020-04-29 10:28:52 helpers:104] Loss: 0.06088229641318321
[NeMo I 2020-04-29 10:28:52 helpers:110] training_batch_top@1:  98.4375
[NeMo I 2020-04-29 10:28:52 callbacks:239] Step time: 0.057554006576538086 seconds
[NeMo I 2020-04-29 10:28:52 callbacks:445] Doing Evaluation ..............................
[NeMo I 2020-04-29 10:28:55 helpers:273] ==========>>>>>>Evaluation Loss TestSet: 0.044208068400621414
[NeMo I 2020-04-29 10:28:55 helpers:275] ==========>>>>>>Evaluation Accuracy Top@1 TestSet: 98.7519
[NeMo I 2020-04-29 10:28:55 callbacks:450] Evaluation time: 2.7680327892303467 seconds
[NeMo I 2020-04-29 10:28:57 callbacks:224] Step: 4425
[NeMo I 2020-04-29 10:28:57 helpers:104] Loss: 0.01991482637822628
[NeMo I 2020-04-29 10:28:57 helpers:110] training_batch_top@1:  99.2188
[NeMo I 2020-04-29 10:28:57 callbacks:239] Step time: 0.052809953689575195 seconds
[NeMo I 2020-04-29 10:28:58 callbacks:224] Step: 4450
[NeMo I 2020-

In [17]:
dur = end - start

In [18]:
dur

308.57925844192505

# Evaluation of incorrectly predicted samples

Given that we have a trained model, which performs reasonably well, lets try to listen to the samples where the model is least confident in its predictions.

For this, we need support of the librosa library.

**NOTE**: The following code depends on librosa. To install it, run the following code block first

!pip install librosa

In [19]:
# lets add a path to the checkpoint dir
model_path = neural_factory.checkpoint_dir

In [20]:
model_path

'.//home/fjia/data/freesound_resampled/quartznet-3x1-vcombo_balanced_sc_bg_combine_test/checkpoints'

## Extract the predictions from the model

We want to possess the actual logits of the model instead of just the final evaluation score, so we use `NeuralFactory.infer(...)` to extract the logits per batch of samples provided.

In [21]:
# --- Inference Only --- #
# We've already built the inference DAG above, so all we need is to call infer().
evaluated_tensors = neural_factory.infer(
    # These are the tensors we want to get from the model.
    tensors=[test_loss, test_decoded, test_labels],
    # checkpoint_dir specifies where the model params are loaded from.
    checkpoint_dir=model_path
    )

[NeMo I 2020-04-29 10:29:01 actions:1493] Restoring JasperEncoder from .//home/fjia/data/freesound_resampled/quartznet-3x1-vcombo_balanced_sc_bg_combine_test/checkpoints/JasperEncoder-STEP-4465.pt
[NeMo I 2020-04-29 10:29:01 actions:1493] Restoring JasperDecoderForClassification from .//home/fjia/data/freesound_resampled/quartznet-3x1-vcombo_balanced_sc_bg_combine_test/checkpoints/JasperDecoderForClassification-STEP-4465.pt
[NeMo I 2020-04-29 10:29:02 actions:734] Evaluating batch 0 out of 109
[NeMo I 2020-04-29 10:29:02 actions:734] Evaluating batch 10 out of 109
[NeMo I 2020-04-29 10:29:02 actions:734] Evaluating batch 20 out of 109
[NeMo I 2020-04-29 10:29:02 actions:734] Evaluating batch 30 out of 109
[NeMo I 2020-04-29 10:29:03 actions:734] Evaluating batch 40 out of 109
[NeMo I 2020-04-29 10:29:03 actions:734] Evaluating batch 50 out of 109
[NeMo I 2020-04-29 10:29:03 actions:734] Evaluating batch 60 out of 109
[NeMo I 2020-04-29 10:29:03 actions:734] Evaluating batch 70 out of 1

## Accuracy calculation

In [22]:
correct_count = 0
total_count = 0

for batch_idx, (logits, labels) in enumerate(zip(evaluated_tensors[1], evaluated_tensors[2])):
    acc = classification_accuracy(
        logits=logits,
        targets=labels,
        top_k=[1]
    )

    # Select top 1 accuracy only
    acc = acc[0]

    # Since accuracy here is "per batch", we simply denormalize it by multiplying
    # by batch size to recover the count of correct samples.
    correct_count += int(acc * logits.size(0))
    total_count += logits.size(0)

logging.info(f"Total correct / Total count : {correct_count} / {total_count}")
logging.info(f"Final accuracy : {correct_count / float(total_count)}")

[NeMo I 2020-04-29 10:29:04 <ipython-input-22-674fb7de9132>:19] Total correct / Total count : 13675 / 13861
[NeMo I 2020-04-29 10:29:04 <ipython-input-22-674fb7de9132>:20] Final accuracy : 0.986581054757954


## Precision Recall F1

In [25]:
# todo test
from typing import List, Optional
def binary_classification_confusion_matrix(logits: torch.Tensor, targets: torch.Tensor, top_k: Optional[List[int]] = None) -> List[float]:
    """
    ]
    [TODO]
    Computes the top-k classification accuracy provided with
    un-normalized logits of a model and ground truth targets.
    If top_k is not provided, defaults to top_1 accuracy.
    If top_k is provided as a list, then the values are sorted
    in ascending order.
    Args:
        logits: Un-normalized logits of a model. Softmax will be
            applied to these logits prior to computation of accuracy.
        targets: Vector of integers which represent indices of class
            labels.
        top_k: Optional list of integers in the range [1, max_classes].
    Returns:
        A list of length `top_k`, where each value represents top_i
        accuracy (i in `top_k`).
    """
    if top_k is None:
        top_k = [1]
    max_k = max(top_k)

    with torch.no_grad():
        true_positive = 0
        false_positive = 0
        _, predictions = logits.topk(max_k, dim=1, largest=True, sorted=True)
        predictions = predictions.t().squeeze()

        # speech(command) positive | background negative
        
        true_negative = 0
        false_negative = 0
        false_positive = 0
        true_positive = 0
        
        for i in range(predictions.size(-1)):
            pred = predictions[i]
            targ = targets[i]
#             print(pred, targ)
            if pred == 0 and targ == 0:
                true_negative += 1
            elif pred == 0 and targ == 1:
                false_negative += 1
            elif pred == 1 and targ == 0:
                false_positive += 1
            elif pred == 1 and targ == 1:
                true_positive += 1
            else:
                raise ValueError('Predictions or targets not in 0/1')
               
                
#         correct = predictions.eq(targets.view(1, -1)).expand_as(predictions)
#         print(correct)

#         results = []
#         for k in top_k:
#             correct_k = correct[:k].view(-1).float().mean().to('cpu').numpy()
#             results.append(correct_k)

#     return results
    return true_negative, false_negative , false_positive, true_positive


In [26]:
correct_count = 0
total_count = 0

total_true_negative, total_false_negative , total_false_positive, total_true_positive = 0, 0, 0, 0
for batch_idx, (logits, labels) in enumerate(zip(evaluated_tensors[1], evaluated_tensors[2])):
    true_negative, false_negative , false_positive, true_positive = binary_classification_confusion_matrix(
        logits=logits,
        targets=labels,
        top_k=[1]
    )

    total_true_negative += true_negative
    total_false_negative += false_negative
    total_false_positive += false_positive
    total_true_positive  += true_positive
    
#     print(cc)

precision = total_true_positive / (total_true_positive + total_false_positive)
recall = total_true_positive / (total_true_positive + total_false_negative)
f1_score =  2 * precision * recall / (precision + recall)
logging.info(f"Final Precision: {precision}")
logging.info(f"Final Recall : {recall}")
logging.info(f"Final F1 score : {f1_score}")

[NeMo I 2020-04-29 10:42:14 <ipython-input-26-c91b90f9f69f>:22] Final Precision: 0.9842262327696462
[NeMo I 2020-04-29 10:42:14 <ipython-input-26-c91b90f9f69f>:23] Final Recall : 0.9892872446793315
[NeMo I 2020-04-29 10:42:14 <ipython-input-26-c91b90f9f69f>:24] Final F1 score : 0.9867502493232654


## Filtering out incorrect samples
Let us now filter out the incorrectly labeled samples from the total set of samples in the test set

In [24]:
import torch
import librosa
import json
import IPython.display as ipd

In [ ]:
# First lets create a utility class to remap the integer class labels to actual string label
class ReverseMapLabel:
    def __init__(self, data_layer: nemo_asr.AudioToSpeechLabelDataLayer):
        self.label2id = dict(data_layer._dataset.label2id)
        self.id2label = dict(data_layer._dataset.id2label)

    def __call__(self, pred_idx, label_idx):
        return self.id2label[pred_idx], self.id2label[label_idx]

In [ ]:
# Next, lets get the indices of all the incorrectly labeled samples
sample_idx = 0
incorrect_preds = []
rev_map = ReverseMapLabel(eval_data_layer)

# Remember, evaluated_tensor = (loss, logits, labels)
for batch_idx, (logits, labels) in enumerate(zip(evaluated_tensors[1], evaluated_tensors[2])):
    probs = torch.softmax(logits, dim=-1)
    probas, preds = torch.max(probs, dim=-1)

    incorrect_ids = (preds != labels).nonzero()
    for idx in incorrect_ids:
        proba = float(probas[idx][0])
        pred = int(preds[idx][0])
        label = int(labels[idx][0])
        idx = int(idx[0]) + sample_idx

        incorrect_preds.append((idx, *rev_map(pred, label), proba))

    sample_idx += labels.size(0)

logging.info(f"Num test samples : {total_count}")
logging.info(f"Num errors : {len(incorrect_preds)}")

# First lets sort by confidence of prediction
incorrect_preds = sorted(incorrect_preds, key=lambda x: x[-1], reverse=True) #False

In [ ]:
# Next, lets get the indices of all the incorrectly labeled samples
sample_idx = 0
correct_preds = []
rev_map = ReverseMapLabel(eval_data_layer)

# Remember, evaluated_tensor = (loss, logits, labels)
for batch_idx, (logits, labels) in enumerate(zip(evaluated_tensors[1], evaluated_tensors[2])):
    probs = torch.softmax(logits, dim=-1)
    probas, preds = torch.max(probs, dim=-1)

    correct_ids = (preds == labels).nonzero()
    for idx in correct_ids:
        proba = float(probas[idx][0])
        pred = int(preds[idx][0])
        label = int(labels[idx][0])
        idx = int(idx[0]) + sample_idx

        correct_preds.append((idx, *rev_map(pred, label), proba))

    sample_idx += labels.size(0)

logging.info(f"Num test samples : {total_count}")
logging.info(f"Num correct : {len(correct_preds)}")

# First lets sort by confidence of prediction
correct_preds = sorted(correct_preds, key=lambda x: x[-1], reverse=True) #False

## Examine a subset of incorrect samples
Lets print out the (test id, predicted label, ground truth label, confidence) tuple of first 20 incorrectly labeled samples

In [ ]:
for incorrect_sample in incorrect_preds[:50]:
    
#     if incorrect_sample[2] == 'background':
#         print(incorrect_sample)
    logging.info(str(incorrect_sample))

##  Define a threshold below which we designate a model's prediction as "low confidence"

In [ ]:
# Filter out how many such samples exist
low_confidence_threshold = 0.55
count_low_confidence = len(list(filter(lambda x: x[-1] <= low_confidence_threshold, incorrect_preds)))
logging.info(f"Number of low confidence predictions : {count_low_confidence}")

In [ ]:
# Filter out how many such samples exist
high_confidence_threshold = 0.99
count_high_confidence = len(list(filter(lambda x: x[-1] >= high_confidence_threshold, incorrect_preds)))
logging.info(f"Number of high confidence predictions : {count_high_confidence}")

# Lets hear the samples which the model has least confidence in !

In [ ]:
# First lets create a helper function to parse the manifest files
def parse_manifest(manifest):
    data = []
    for line in manifest:
        line = json.loads(line)
        data.append(line)

    return data

In [ ]:
# Next, lets create a helper function to actually listen to certain samples
def listen_to_file(sample_id, pred=None, label=None, proba=None):
    # Load the audio waveform using librosa
    filepath = test_samples[sample_id]['audio_filepath']
    if 'offset' in test_samples[sample_id]:
        audio, sample_rate = librosa.load(filepath,
                                          offset = test_samples[sample_id]['offset'],
                                          duration = test_samples[sample_id]['duration'])
    else:
         audio, sample_rate = librosa.load(filepath)

    if pred is not None and label is not None and proba is not None:
        logging.info(f"filepath: {filepath}, Sample : {sample_id} Prediction : {pred} Label : {label} Confidence = {proba: 0.4f}")
    else:
        logging.info(f"Sample : {sample_id}")

    return ipd.Audio(audio, rate=sample_rate)


In [ ]:
# Now lets load the test manifest into memory

all_test_samples = []
for _ in test_dataset.split(','):
    with open(_, 'r') as test_f:
        test_samples = test_f.readlines()
        print(_, len(test_samples))
       
        all_test_samples.extend(test_samples)
        
test_samples = parse_manifest(all_test_samples)
print(len(test_samples))

In [ ]:
incorrect_preds

In [ ]:
# Finally, lets listen to all the audio samples where the model made a mistake
# Note: This list of incorrect samples may be quite large, so you may choose to subsample `incorrect_preds`

for sample_id, pred, label, proba in incorrect_preds[:200]:
    filepath = test_samples[sample_id]['audio_filepath']
    
    print(test_samples[sample_id])
#     if filepath not in exist:
    ipd.display(listen_to_file(sample_id, pred=pred, label=label, proba=proba))
    exist.add(filepath)